In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import nept

In [ ]:
event_labels = dict(recording_start='Starting Recording',
                    recording_stop='Stopping Recording',
                    north_feeder='TTL Output on AcqSystem1_0 board 0 port 0 value (0x0010).',
                    north_pb='TTL Input on AcqSystem1_0 board 0 port 1 value (0x0010).',
                    east_feeder='TTL Output on AcqSystem1_0 board 0 port 0 value (0x0020).',
                    east_pb='TTL Input on AcqSystem1_0 board 0 port 1 value (0x0020).',
                    south_feeder='TTL Output on AcqSystem1_0 board 0 port 0 value (0x0040).',
                    south_pb='TTL Input on AcqSystem1_0 board 0 port 1 value (0x0040).',
                    trial_start='trial start',
                    trial_end='trial end')

In [ ]:
thisdir = os.getcwd()
output_filepath = os.path.join(thisdir, "plots")
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

# rat = "R191"
rat = "R192"
data_filepath = os.path.join("E:\\data\\data-experience\\", rat, "RR3")
# data_filepath = os.path.join("/home/emily/data/", rat)

In [ ]:
n_trials = []
mean_durations = []
for filename in os.listdir(data_filepath):
    if filename.endswith("behavior"):
        print(filename)
        date = filename[5:15]

        event_filepath = os.path.join(data_filepath, filename, date+"_Events.nev")
        events = nept.load_events(event_filepath, event_labels)
        
        print(events["recording_start"])
        print(events["recording_stop"])

        if events["recording_start"][1] < events["trial_start"][0]:
            starts = np.insert(events["trial_start"], 0, events["recording_start"][1])
            stops = events["trial_end"]
        else:
            starts =events["trial_start"]
        stops = events["trial_end"]
        print(len(starts), len(stops))
        print(starts[:5], stops[:5])
        if len(starts) > len(stops):
            starts = starts[:-1]
        if len(starts) < len(stops):
            stops = stops[:-1]
        if stops[-1]-starts[-1] > 0:
            trials = nept.Epoch(starts, stops)
        else:
            trials = nept.Epoch(stops, starts)

        n_trials.append(trials.n_epochs)
        mean_durations.append(np.mean(trials.durations))
        print("Duration:", np.mean(trials.durations))
        print(trials.durations)
#         1/0

In [ ]:
mean_durations

In [ ]:
fig, ax = plt.subplots()
x_ticks = np.arange(len(n_trials))+1
ax.plot(x_ticks, n_trials, lw=2)
ax.set_ylim(0, 82)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

plt.xlabel("Day")
plt.ylabel("Number of trials")
plt.title(rat+ " T-maze in RR3")
ax.set_xticks(x_ticks)
# plt.savefig(os.path.join(output_filepath, rat+"_n-trials.png"))
# plt.close()
plt.show()

In [ ]:
fig, ax = plt.subplots()
x_ticks = np.arange(len(mean_durations))+1
ax.plot(x_ticks, mean_durations, lw=2)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

plt.xlabel("Day")
plt.ylabel("Mean trial duration (s)")
plt.title(rat + " T-maze in RR3")
ax.set_xticks(x_ticks)
# plt.savefig(os.path.join(output_filepath, rat+"_mean-durations.png"))
# plt.close()
plt.show()

In [ ]:
x_ticks = np.arange(len(mean_durations))+1

durations = mean_durations
trial_ns = n_trials

fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('Session')
ax1.set_ylabel('Number of trials', color=color)
ax1.plot(x_ticks, n_trials, color=color)
ax1.set_ylim(0, 82)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('Mean trial duration (s)', color=color)  # we already handled the x-label with ax1
ax2.plot(x_ticks, mean_durations, color=color)
ax2.tick_params(axis='y', labelcolor=color)

ax1.spines['top'].set_visible(False)
ax2.spines['top'].set_visible(False)

plt.title("T-maze in RR3\n(Rat 2)")

fig.tight_layout()
plt.savefig(os.path.join(output_filepath, rat+"_tmaze.png"))
plt.close()
# plt.show()